In [1]:
import ctypes as ctypes
import ctypes.wintypes
import sys
import time

import ctypes_wrappers


HWND_MESSAGE = -3

WM_QUIT = 0x0012
WM_INPUT = 0x00FF
WM_KEYUP = 0x0101
WM_CHAR = 0x0102

HID_USAGE_PAGE_GENERIC = 0x01

RIDEV_NOLEGACY = 0x00000030
RIDEV_INPUTSINK = 0x00000100
RIDEV_CAPTUREMOUSE = 0x00000200

RID_HEADER = 0x10000005
RID_INPUT = 0x10000003

RIM_TYPEMOUSE = 0
RIM_TYPEKEYBOARD = 1
RIM_TYPEHID = 2

PM_NOREMOVE = 0x0000

In [ ]:
def wnd_proc(hwnd, msg, wparam, lparam):
    print(
        f"Handle message - hwnd: 0x{hwnd:016X} msg: 0x{msg:08X} wp: 0x{wparam:016X} lp: 0x{lparam:016X}"
    )
    if msg == WM_INPUT:
        size = ctypes.wintypes.UINT(0)
        res = ctypes_wrappers.GetRawInputData(
            ctypes.cast(lparam, ctypes_wrappers.PRAWINPUT),
            RID_INPUT,
            None,
            ctypes.byref(size),
            ctypes.sizeof(ctypes_wrappers.RAWINPUTHEADER),
        )
        if res == ctypes.wintypes.UINT(-1) or size == 0:
            print_error(text="GetRawInputData 0")
            return 0
        buf = ctypes.create_string_buffer(size.value)
        res = ctypes_wrappers.GetRawInputData(
            ctypes.cast(lparam, ctypes_wrappers.PRAWINPUT),
            RID_INPUT,
            buf,
            ctypes.byref(size),
            ctypes.sizeof(ctypes_wrappers.RAWINPUTHEADER),
        )
        if res != size.value:
            print_error(text="GetRawInputData 1")
            return 0
        # print("kkt: ", ctypes.cast(lparam, ctypes_wrappers.PRAWINPUT).contents.to_string())
        ri = ctypes.cast(buf, ctypes_wrappers.PRAWINPUT).contents
        # print(ri.to_string())
        head = ri.header
        print(head.to_string())
        # print(ri.data.mouse.to_string())
        # print(ri.data.keyboard.to_string())
        # print(ri.data.hid.to_string())
        if head.dwType == RIM_TYPEMOUSE:
            data = ri.data.mouse
        elif head.dwType == RIM_TYPEKEYBOARD:
            data = ri.data.keyboard
            if data.VKey == 0x1B:
                ctypes_wrappers.PostQuitMessage(0)
        elif head.dwType == RIM_TYPEHID:
            data = ri.data.hid
        else:
            print("Wrong raw input type!!!")
            return 0
        print(data.to_string())
    return ctypes_wrappers.DefWindowProc(hwnd, msg, wparam, lparam)

In [ ]:
wnd_cls = "SO049572093_RawInputWndClass"
wcx = ctypes_wrappers.WNDCLASSEX()
wcx.cbSize = ctypes.sizeof(ctypes_wrappers.WNDCLASSEX)
# wcx.lpfnWndProc = ctypes.cast(ctypes_wrappers.DefWindowProc, ctypes.c_void_p)
wcx.lpfnWndProc = ctypes_wrappers.WNDPROC(wnd_proc)
wcx.hInstance = ctypes_wrappers.GetModuleHandle(None)
wcx.lpszClassName = wnd_cls
# print(dir(wcx))


res = ctypes_wrappers.RegisterClassEx(ctypes.byref(wcx))
if not res:
    print_error(text="RegisterClass")
    return 0
hwnd = ctypes_wrappers.CreateWindowEx(
    0, wnd_cls, None, 0, 0, 0, 0, 0, 0, None, wcx.hInstance, None
)
if not hwnd:
    print_error(text="CreateWindowEx")
    return 0
# print("hwnd:", hwnd)
if not register_devices(hwnd):
    return 0
msg = ctypes.wintypes.MSG()
pmsg = ctypes.byref(msg)
print("Start loop (press <ESC> to exit)...")
while res := ctypes_wrappers.GetMessage(pmsg, None, 0, 0):
    if res < 0:
        print_error(text="GetMessage")
        break
    ctypes_wrappers.TranslateMessage(pmsg)
    ctypes_wrappers.DispatchMessage(pmsg)

In [ ]:
def wnd_proc(hwnd, msg, wparam, lparam):
    print(
        f"Handle message - hwnd: 0x{hwnd:016X} msg: 0x{msg:08X} wp: 0x{wparam:016X} lp: 0x{lparam:016X}"
    )
    if msg == WM_INPUT:
        size = ctypes.wintypes.UINT(0)
        res = ctypes_wrappers.GetRawInputData(
            ctypes.cast(lparam, ctypes_wrappers.PRAWINPUT),
            RID_INPUT,
            None,
            ctypes.byref(size),
            ctypes.sizeof(ctypes_wrappers.RAWINPUTHEADER),
        )

        buf = ctypes.create_string_buffer(size.value)
        res = ctypes_wrappers.GetRawInputData(
            ctypes.cast(lparam, ctypes_wrappers.PRAWINPUT),
            RID_INPUT,
            buf,
            ctypes.byref(size),
            ctypes.sizeof(ctypes_wrappers.RAWINPUTHEADER),
        )

        # print("kkt: ", ctypes.cast(lparam, ctypes_wrappers.PRAWINPUT).contents.to_string())
        ri = ctypes.cast(buf, ctypes_wrappers.PRAWINPUT).contents
        # print(ri.to_string())
        head = ri.header
        print(head.to_string())
        # print(ri.data.mouse.to_string())
        # print(ri.data.keyboard.to_string())
        # print(ri.data.hid.to_string())
        if head.dwType == RIM_TYPEMOUSE:
            data = ri.data.mouse
        elif head.dwType == RIM_TYPEKEYBOARD:
            data = ri.data.keyboard
            if data.VKey == 0x1B:
                ctypes_wrappers.PostQuitMessage(0)
        elif head.dwType == RIM_TYPEHID:
            data = ri.data.hid

        print(data.to_string())
    return ctypes_wrappers.DefWindowProc(hwnd, msg, wparam, lparam)